In [1]:
import numpy as np
import matplotlib.pyplot as plt
from src.utils import tensorprod, conj_tp, bra_ket, expectation
from src.gates import CX, H, rx, ry, rz, cry, p

In [2]:
I = np.eye(2)

In [3]:
Zero = np.array([1, 0]).reshape(-1, 1)
One = np.array([0, 1]).reshape(-1, 1)  

In [4]:
def Ansatz(x):
    # theta = np.array(theta)    
    init  = tensorprod(Zero, Zero, Zero, Zero)
    l_1  = tensorprod(H, H, H, H)
    l_2 = tensorprod(p(2 * x[0]), p(2 * x[1]), p(2 * x[2]), p(2 * x[3]))
    l_3 = tensorprod(CX, CX)
    l_4 = tensorprod(I, p(2 * (np.pi-x[0]) * (np.pi-x[1])), I, p(2 * (np.pi-x[2]) * (np.pi-x[3])))
    l_5 = tensorprod(CX, CX)
    l_6 = tensorprod(I, CX, I)
    l_7 = tensorprod(I, I, p(2 * (np.pi-x[1]) * (np.pi-x[2])), I)
    l_8 = tensorprod(I, CX, I)
    # l_1 = tensorprod(rx(theta[0]), I_)
    # l_2 = cry(theta[1])
    return l_8 @ l_7 @ l_6 @ l_5 @ l_4 @ l_3 @ l_2 @ l_1 @ init

In [5]:
p(np.pi) @ p(np.pi)

array([[ 1.+0.0000000e+00j, -0.+0.0000000e+00j],
       [-0.+0.0000000e+00j,  1.-2.4492936e-16j]])

In [6]:
tensorprod(p(np.pi), p(np.pi)).shape

(4, 4)

In [7]:
from sklearn.datasets import make_classification
X, y  = make_classification(1000, 4, random_state=137)

In [8]:
X[0]

array([-0.03639866,  0.54293503, -1.15013647,  0.99478782])

In [14]:
%%timeit 
Ansatz(X[0])

424 µs ± 13.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
np.conj(Ansatz(X[0])).reshape(-1) @ Ansatz(X[0])

array([1.+0.j])

In [17]:
Ansatz(X[0])

array([[ 0.25      +0.j        ],
       [ 0.00094174+0.24999823j],
       [ 0.18577687+0.16729302j],
       [-0.24999836-0.00090584j],
       [-0.14898864+0.20075454j],
       [-0.20131435-0.14823135j],
       [-0.16910788+0.18412639j],
       [ 0.00335888-0.24997743j],
       [-0.18524595-0.16788073j],
       [ 0.16718172-0.18587703j],
       [-0.02531656-0.24871484j],
       [ 0.18463644+0.16855084j],
       [-0.06025166-0.24263087j],
       [ 0.24240218-0.06116522j],
       [-0.03462587-0.24759049j],
       [ 0.19082743+0.16150817j]])

# Qiskit

In [23]:
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import ZZFeatureMap

In [21]:
zf = ZZFeatureMap(4, reps=1, entanglement='pairwise')

In [22]:
zf.assign_parameters(dict(
        zip(zf.parameters, X[0][::-1])
    ), inplace=True
)

In [46]:
zf.decompose().draw()

┌───┐ ┌─────────────────────┐                                        »
q_0: ┤ H ├─┤ P(1.98957564792519) ├────■───────────────────────────■───────»
     ├───┤ ├─────────────────────┤  ┌─┴─┐┌─────────────────────┐┌─┴─┐     »
q_1: ┤ H ├─┤ P(-2.3002729447698) ├──┤ X ├┤ P(18.4270096301884) ├┤ X ├──■──»
     ├───┤ ├─────────────────────┤  └───┘└─────────────────────┘└───┘┌─┴─┐»
q_2: ┤ H ├─┤ P(1.08587005811716) ├────■───────────────────────────■──┤ X ├»
     ├───┤┌┴─────────────────────┴─┐┌─┴─┐┌─────────────────────┐┌─┴─┐└───┘»
q_3: ┤ H ├┤ P(-0.0727973288450728) ├┤ X ├┤ P(16.5170227384935) ├┤ X ├─────»
     └───┘└────────────────────────┘└───┘└─────────────────────┘└───┘     »
«                                 
«q_0: ────────────────────────────
«                                 
«q_1: ─────────────────────────■──
«     ┌─────────────────────┐┌─┴─┐
«q_2: ┤ P(22.3054692312736) ├┤ X ├
«     └─────────────────────┘└───┘
«q_3: ────────────────────────────
«

In [24]:
state = Statevector.from_int(0, 2**4).evolve(zf)

In [32]:
state = np.array(state).reshape(-1, 1)

In [43]:
np.isclose(state, Ansatz(X[0]))

array([[ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True]])